# 220228 과제
method1_entropy = (E(all to 1) - E(all to 0)) * entropy <br>

entropy는 기존꺼 써도 됨. [4-1]에서 만들었었음.<br>
[Method1_concat_all.csv]에 저장돼있음.

# library

In [1]:
#데이터 분석 라이브러리
import numpy as np
print("numpy version: {}". format(np.__version__))

import pandas as pd

#시각화 라이브러리
import matplotlib as mpl
import matplotlib.pyplot as plt
print("matplotlib version: {}". format(mpl.__version__))

import seaborn as sns
print("seaborn version: {}". format(sns.__version__))

#딥러닝 라이브러리
import tensorflow as tf

#기타 라이브러리
import os
import random
import time

#경고 에러 무시
import warnings
warnings.filterwarnings('ignore')
print('-'*50)

#시간 확인을 용이하게 해주는 라이브러리
from tqdm import tqdm

#결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from varname import nameof

numpy version: 1.19.5
matplotlib version: 3.4.2
seaborn version: 0.11.1
--------------------------------------------------


# GPU

In [2]:
#https://www.tensorflow.org/guide/gpu#allowing_gpu_memory_growth
#프로세스의 요구량만큼 메모리 사용 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        for i in range(len(gpus)):
            tf.config.experimental.set_memory_growth(gpus[i], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)
        
#지정해서 쓰고 싶으면 model 쓸 때는 with 구문 안에 넣기

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


# Data Load

In [7]:
pd.read_csv('Method1_concat_all.csv')
entropy = pd.read_csv('Method1_concat_all.csv')['Entropy']

,feature,baseline_bce,bce,diff,ReLU,preds_mean,preds2_mean,diff_preds,Sign,Entropy,Method1
0,0,1.014362,1.009931,-0.004431,0.000000,0.633561,0.613608,-0.019953,-1.0,0.894548,-0.000000
1,50803,1.014362,1.016480,0.002118,0.002118,0.633561,0.607474,-0.026087,-1.0,0.033443,-0.000071
2,50804,1.014362,1.017856,0.003495,0.003495,0.633561,0.666301,0.032740,1.0,0.484874,0.001694
3,50805,1.014362,1.015937,0.001575,0.001575,0.633561,0.599635,-0.033926,-1.0,0.001172,-0.000002
4,50806,1.014362,1.019767,0.005405,0.005405,0.633561,0.682195,0.048633,1.0,0.074665,0.000404
...,...,...,...,...,...,...,...,...,...,...,...
4063,78112001103,1.014362,1.016376,0.002014,0.002014,0.633561,0.583690,-0.049871,-1.0,0.036891,-0.000074
4064,79511050204,1.014362,1.014257,-0.000105,0.000000,0.633561,0.631427,-0.002134,-1.0,-0.000000,0.000000
4065,87701071218,1.014362,1.027771,0.013409,0.013409,0.633561,0.722150,0.088589,1.0,0.057111,0.000766
4066,87701083336,1.014362,1.014223,-0.000139,0.000000,0.633561,0.640804,0.007243,1.0,0.011373,0.000000


In [6]:
entropy

0       0.894548
1       0.033443
2       0.484874
3       0.001172
4       0.074665
          ...   
4063    0.036891
4064   -0.000000
4065    0.057111
4066    0.011373
4067    0.003458
Name: Entropy, Length: 4068, dtype: float64

In [9]:
# 1) DATA 
import random    
# ---------------------
seed_num = 42
# ---------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')


# 2) MODEL
# with tf.device('/device:GPU:0'): #모델 돌리는 동안 다 이 안에 넣어놓기
#     tf.config.experimental.set_virtual_device_configuration(gpus[1], 
#                                                             [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5*1024)])

from keras.models import load_model
lstm2 = load_model('/project/LSH/model/allfit_ep500/allfit_ep500_seed42-17-0.7619.hdf5')


# 3) all_to_1
# FEATURES
a = pd.read_csv('/project/LSH/total_data_7727.csv')
features = list(a['ITEMID'].sort_values().unique()) # 4068개 ITEMID 


preds2_entropy_dic = {} # method1_entropy
preds2_dic = {} # method1_부록
# preds_mean_lst = []

for k in tqdm(range(len(features))):

    # REVERSE ALL FEATURE K
    save_col = x[:,:,k].copy()

    # all_to_1
    x[:,:,k] = 1

    # COMPUTE pred WITH FEATURE K all_to_1, preds2_mean_list
    preds2_1 = lstm2.predict(x, batch_size = 10000, workers = -1, use_multiprocessing = True)
    mean_preds2_1 = np.mean(preds2_1)

    # all_to_0
    x[:,:,k] = 0 #어차피 all change라서 save_col 안하고 해도 됨.

    # COMPUTE pred WITH FEATURE K all_to_0, preds2_mean_list
    preds2_0 = lstm2.predict(x, batch_size = 10000, workers = -1, use_multiprocessing = True)
    mean_preds2_0 = np.mean(preds2_0)
    
    diff = mean_preds2_1 - mean_preds2_0 # 값 한개
    
    preds2_entropy_dic[features[k]] = diff * entropy[k]
    preds2_dic[features[k]] = diff # 어차피 아래랑 같은 값 들어감.
#     preds_mean_lst.append(diff)

    x[:,:,k] = save_col

#     break

100%|██████████| 4068/4068 [2:19:23<00:00,  2.06s/it]  


In [10]:
dic1 = preds2_entropy_dic.copy()
dic2 = preds2_dic.copy()

dic1
dic2

{0: -0.027002115725733303,
 50803: -0.00029644816817439557,
 50804: 0.008487710189780675,
 50805: -0.000111375359066917,
 50806: 0.0011474966125577058,
 50808: 0.009668338544139427,
 50809: 0.007934872443005383,
 50811: 0.0032995537353701808,
 50813: 0.013346451458146958,
 50814: -0.00013348186342126433,
 50818: -0.006972436933230711,
 50820: 0.018014796531599073,
 50821: 0.003727659731449496,
 50822: -0.004684145160290563,
 50824: 0.0005442968141361923,
 50852: -0.0004807911589301673,
 50853: -6.17581148467484e-05,
 50856: -0.0019499727832578133,
 50858: 0.0,
 50861: -0.05002573854908195,
 50862: 0.019732943089730223,
 50863: -0.0017302116520637365,
 50864: 0.00010633638738350214,
 50865: 3.2869041839193886e-05,
 50866: 0.0010469769887782234,
 50867: -0.012069224812599307,
 50868: 0.01866052742400768,
 50871: 0.0,
 50872: -0.0,
 50873: 6.4346265298489415e-06,
 50876: -1.1334881030836546e-05,
 50877: -6.026651839120325e-06,
 50878: -0.014081573011501036,
 50881: -3.710843109940858e-05,

{0: -0.030185223,
 50803: -0.008864403,
 50804: 0.01750499,
 50805: -0.09501827,
 50806: 0.0153687,
 50808: 0.028248787,
 50809: 0.02590388,
 50811: 0.018739402,
 50813: 0.044186175,
 50814: -0.09906942,
 50818: -0.011807382,
 50820: 0.03232968,
 50821: 0.0053799152,
 50822: -0.032090783,
 50824: 0.0072054863,
 50852: -0.016738355,
 50853: -0.03323549,
 50856: -0.2595349,
 50858: -0.0021637082,
 50861: -0.12506282,
 50862: 0.041935682,
 50863: -0.004237354,
 50864: 0.09071934,
 50865: 0.013076901,
 50866: 0.04412031,
 50867: -0.102066636,
 50868: 0.06351441,
 50871: -0.0018833876,
 50872: 0.00035238266,
 50873: 0.01029706,
 50876: -0.049537957,
 50877: -0.026338875,
 50878: -0.031239271,
 50881: -0.019970119,
 50882: 0.041732848,
 50883: -0.009365439,
 50885: 0.0071076155,
 50889: -0.119093,
 50890: -0.18015352,
 50891: -0.09111345,
 50892: 0.039910972,
 50893: 0.042416096,
 50894: -0.001106739,
 50895: -0.021498919,
 50896: -0.046100676,
 50898: -0.0020228624,
 50899: -0.05753821,
 50

In [11]:
df1 = pd.DataFrame(dic1.items(), columns=['feature', 'method1_entropy'])
df1

df2 = pd.DataFrame(dic2.items(), columns=['feature', 'method1_부록']) #hp columns는 column name 설정 
df2

,feature,method1_entropy
0,0,-0.027002
1,50803,-0.000296
2,50804,0.008488
3,50805,-0.000111
4,50806,0.001147
...,...,...
4063,78112001103,-0.002549
4064,79511050204,0.000000
4065,87701071218,0.002658
4066,87701083336,0.000334


,feature,method1_부록
0,0,-0.030185
1,50803,-0.008864
2,50804,0.017505
3,50805,-0.095018
4,50806,0.015369
...,...,...
4063,78112001103,-0.069097
4064,79511050204,-0.001821
4065,87701071218,0.046544
4066,87701083336,0.029366


In [12]:
df1.to_csv('final_Method1_entropy.csv', index = False)
df2.to_csv('final_Method1_부록.csv', index = False)

In [16]:
df1.sort_values('method1_entropy', ascending = False)[:10]
df2.sort_values('method1_부록', ascending = False)[:10]

,feature,method1_entropy
206,51277,0.078080
121,51006,0.075320
41,50893,0.037807
56,50912,0.036471
2148,409176230,0.032615
34,50882,0.032041
184,51244,0.028505
2340,536338101,0.026930
119,51003,0.024043
1733,173024256,0.022380


,feature,method1_부록
2195,409606211,0.245111
1276,74606211,0.234800
1706,172438110,0.219769
3283,51479005520,0.203799
157,51148,0.200978
1081,67434504,0.198929
2403,591319872,0.194788
1426,83000730,0.192208
2024,338068704,0.190258
982,54817525,0.184953
